In [2]:
import os
import subprocess
import numpy as np
import pandas as pd
import mlflow
import matplotlib.pyplot as plt
import scikitplot as skplt
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier

## Initiate MLFlow

In [3]:
bashcommand = "lpass show -p Shared-Preste-ALL/MlFlow"
pwd = subprocess.getoutput(bashcommand)

os.environ['MLFLOW_TRACKING_USERNAME'] = 'preste'
os.environ['MLFLOW_TRACKING_PASSWORD'] = pwd

remote_server_uri = "https://mlflow.preste.ai"
mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name="macrophages")

In [ ]:
# Hyperparameter grid for each model
param_knn = { 'n_neighbors' : [5,7,9,11,13,15,25,40],
            'weights' : ['uniform','distance'],
            'metric' : ['minkowski','euclidean','manhattan']
            }
param_lreg = {'C' : np.logspace(-4, 4, 50),
            'penalty' : ['l1', 'l2', 'elasticnet'],
            'solver' : ['lbfgs','newton-cg','liblinear'],
             'max_iter' : [15000]
             }
param_svm = {'C': [0.1, 1, 10, 100, 1000], 
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'kernel': ['rbf']
            } 
param_rf = {'max_depth':[3,5,10,None],
            'n_estimators':[10,100,200],
            'max_features':[1,3,5,7],
            'min_samples_leaf':[1,2,3],
            'min_samples_split':[1,2,3],
            'warm_start' : [True]
           }
param_gbc = {learning_rate": [0.01, 0.05,0.1,0.2],
            "min_samples_split": np.linspace(0.1, 0.5, 4),
            "min_samples_leaf": np.linspace(0.1, 0.5, 4),
            "max_depth":[3,5,8],
            "max_features":["log2","sqrt"],
            "criterion": ["friedman_mse",  "mae"],
            "subsample":[0.5, 0.8, 1.0]
            }
params_xgb = {'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0],
            'min_child_weight': [1, 2, 3],
            'gamma': [0, 0.1, 0.2],
            'scale_pos_weight': [1, 2, 3]
            }

# Models and corresponding hyperparameter grids
models = {"KNN": [KNeighborsClassifier(), param_knn],
        "Logistic Regression": [LogisticRegression(), param_lreg],     
        "RandomForestClassifier": [RandomForestClassifier(), param_rf],
        "SVM": [SVC(), param_svm],
        "GBC": [GradientBoostingClassifier(), param_gbc],
        "XGB": [XGBClassifier(), params_xgb]}

In [5]:
preprocessed_data_path = '../../20x_datasets/Preprocessed_data/'
nucleus_dataset = pd.read_csv(preprocessed_data_path + 'clean_20x_CD86_P4G4_nucleus_dataset.csv')

X_nucleus = nucleus_dataset.drop('CellType', axis=1)
y_nucleus = nucleus_dataset['CellType']

X_train, X_test, y_train, y_test = train_test_split(X_nucleus, y_nucleus, test_size = 0.25, stratify = y_nucleus, random_state=0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=666)
scores = []
scoring = {"Accuracy": 'accuracy', 'F1_micro': 'f1_micro', 'Log_loss': 'neg_log_loss'}


for model_param in models.values():
    # run grid search
    model_name = model_param[0]
    model_grid = GridSearchCV(estimator=model_name, param_grid=model_param[1], cv=kf, scoring=scoring, refit="Accuracy")
    model_grid.fit(X_train_scaled, y_train)
    best_params = model_grid.best_params_
    best_model = model_name.set_params(**best_params)
    
    # Extract the results of the grid search
    cv_results = pd.DataFrame(model_grid.cv_results_)
    best_model_results = cv_results[cv_results['rank_test_Accuracy'] == 1]
    
    # Infer the model signature
    best_model.fit(X_train_scaled, y_train)
    y_pred = best_model.predict(X_test_scaled)
    signature = infer_signature(X_test_scaled, y_pred)
    
    # add model to MLflow registry
    with  mlflow.start_run(run_name=str(model_name)):
        
        # Log model params and metrics
        mlflow.log_params(best_params)
        metrics = {
            'accuracy': best_model_results['mean_test_Accuracy'].values[0],
            'accuracy_std': best_model_results['std_test_Accuracy'].values[0],
            'F1_micro' : best_model_results['mean_test_F1_micro'].values[0],
            'F1_micro_std' : best_model_results['std_test_F1_micro'].values[0],
            'Log_loss' : best_model_results['mean_test_Log_loss'].values[0],
            'Log_loss_std' : best_model_results['std_test_Log_loss'].values[0],
            'Mean_score_time' : best_model_results['mean_score_time'].values[0]
            }
        mlflow.log_metrics(metrics)
        
        # add model to registry
        mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name=str(model_name),
    )
    mlflow.end_run()
    
    print(model_name, "highest score is", best_model_results['mean_test_Accuracy'].values[0])
    print(best_params)

c:\_\jupyter_3\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:38:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:38:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\_\jupyter_3\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:38:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:38:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\_\jupyter_3\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

c:\_\jupyter_3\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:39:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:39:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\_\jupyter_3\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:39:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:39:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 